In [1]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey, select
from sqlalchemy.orm import relationship

## Create the NASDAQ database

In [2]:
# Create the NADAQ database and connect to it
engine = create_engine('sqlite:///nasdaq.db')

In [3]:
# Sets an object to utilize the default declarative base
Base = declarative_base()

In [4]:
# Create classes
class Metadata(Base):
    __tablename__ = 'metadata'
    ticker_id = Column(Integer, primary_key=True, unique=True)
    symbol = Column(String(5))
    asset_type = Column(String(20))
    name = Column(String(50))
    description = Column(String(5000))
    currency = Column(String(3))
    country = Column(String(3))
    sector = Column(String(500))
    industry = Column(String(500))
    market_capitalization = Column(Integer)
    ebitda = Column(Integer)
    book_value = Column(Float)
    dividend_per_share = Column(Float)
    dividend_yield = Column(Float)
    figures = relationship("Data", back_populates="ticker")
    
class Data(Base):
    __tablename__ = 'data'
    id = Column(Integer, primary_key=True, unique=True)
    ticker_id = Column(Integer, ForeignKey("metadata.ticker_id"))
    date = Column(String(10))
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Integer)
    ticker = relationship("Metadata", back_populates="figures")
    

In [5]:
# engine = create_engine("sqlite:///nasdaq.sqlite")
# conn = engine.connect()

In [6]:
# Upload the classes to the database through the engine
Base.metadata.create_all(engine)

## Inspect the NASDAQ database

In [7]:
# Import the inspector dependency
from sqlalchemy import inspect

In [8]:
# engine = create_engine("sqlite:///nasdaq.sqlite")

In [9]:
# # Create the inspector and connect it to the database engine
inspector = inspect(engine)

In [10]:
# Check tables exist by getting their names
inspector.get_table_names()

['data', 'metadata']

In [11]:
# Check columns and data types in the Metadata table
columns = inspector.get_columns('metadata')
for column in columns:
    print(column["name"], column["type"])

ticker_id INTEGER
symbol VARCHAR(5)
asset_type VARCHAR(20)
name VARCHAR(50)
description VARCHAR(5000)
currency VARCHAR(3)
country VARCHAR(3)
sector VARCHAR(500)
industry VARCHAR(500)
market_capitalization INTEGER
ebitda INTEGER
book_value FLOAT
dividend_per_share FLOAT
dividend_yield FLOAT


In [12]:
# Check columns and data types in the Data column
columns = inspector.get_columns('data')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
ticker_id INTEGER
date VARCHAR(10)
open FLOAT
high FLOAT
low FLOAT
close FLOAT
volume INTEGER


## Import data into the database

In [13]:
# Import pandas dependency
import pandas as pd

In [14]:
# Import csv file with all the metadata and turn into dataframe
meta_table = pd.read_csv("stockdata/cleaned_metadata.csv", encoding='UTF-8')
meta_table.head()

,ticker_id,symbol,asset_type,name,description,currency,country,sector,industry,market_capitalization,ebitda,book_value,dividend_per_share,dividend_yield
0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,2804149912000,123957002000,3.852,0.93,0.0054
1,1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2510813135000,102022996000,27.750,2.72,0.0080
2,2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",USD,USA,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,1476477321000,63431000000,16.350,0.00,0.0000
3,3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,1115896676000,12379000000,11.050,0.16,0.0004
4,4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",793474040000,44914000000,51.590,0.00,0.0000


In [15]:
# Upload dataframe to the Metadata table in the database
meta_table.to_sql("metadata", engine, if_exists="replace", index=False)

101

In [16]:
# Import csv file with all the trading data and turn into dataframe 
data_table = pd.read_csv("stockdata/clean_data.csv", encoding='UTF-8')
data_table.head()

,id,ticker_id,date,open,high,low,close,volume
0,0,0,2023-09-12,189.485,189.98,173.5400,176.30,500216459
1,1,0,2023-08-31,196.235,196.73,171.9600,187.87,1323817340
2,2,0,2023-07-31,193.780,198.23,186.6000,196.45,996368613
3,3,0,2023-06-30,177.700,194.48,176.9306,193.97,1297863403
4,4,0,2023-05-31,169.280,179.35,164.3100,177.25,1275052503


In [17]:
# # Upload dataframe to the Data table in the database
data_table.to_sql("data", engine, if_exists="replace", index=False)

5940

In [18]:
# from sqlalchemy.ext.automap import automap_base
# Base = automap_base()
# engine = create_engine('sqlite:///nasdaq.db')
# Base.prepare(autoload_with=engine)

In [19]:
# Base.classes.keys()

[]

## Run basic queries to check data imported correctly

In [18]:
# Import the Session dependency and connet through engine
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [19]:
# Get ticker symbols for eacj ticker in the Metadata table
tickers = session.query(Metadata)
for ticker in tickers:
    print(ticker.symbol)

AAPL
MSFT
AMZN
NVDA
META
GOOGL
GOOG
TSLA
AVGO
ADBE
COST
PEP
CSCO
NFLX
CMCSA
AMD
TMUS
INTC
INTU
TXN
AMGN
AMAT
HON
QCOM
BKNG
SBUX
ISRG
ADP
MDLZ
GILD
LRCX
REGN
ADI
VRTX
MU
PANW
MELI
SNPS
KLAC
PYPL
CDNS
CHTR
MAR
CSX
ABNB
MNST
ORLY
PDD
ASML
NXPI
WDAY
CTAS
FTNT
LULU
MRVL
ADSK
KDP
ODFL
PCAR
PAYX
MCHP
CPRT
ON
MRNA
ROST
KHC
DXCM
AZN
EXC
AEP
IDXX
SGEN
CRWD
BIIB
BKR
TTD
CTSH
VRSK
CEG
CSGP
EA
GFS
XEL
TEAM
FAST
GEHC
DDOG
FANG
WBD
ANSS
DLTR
ALGN
ILMN
ZS
EBAY
WBA
ZM
SIRI
ENPH
JD
LCID


In [27]:
# Get volume for each ticker in the Data table
trades = session.query(Data)
for trade in trades:
    print(trade.volume)

500216459
1323817340
996368613
1297863403
1275052503
967580718
1520461315
1307294493
1443652725
1675731304
1724948219
1868381344
2083968147
1509735435
1447364668
1749297959
2400305601
1683547838
2171827429
1628570227
2106513962
2443227128
1688864233
1565079040
1797948421
1462773381
1916751489
1606114354
1711935110
1889956694
2650845211
1825486961
2239366098
2319687808
2122724412
2895317580
3886793083
1184207050
755162226
810900890
701660022
816530808
1570331732
755223231
734044103
598871365
448922253
621478768
547408488
683515746
473957094
515218768
739456573
506117812
650981384
472540723
828099179
898917007
961321947
789748068
678972040
64463937
108585952
104767910
120963450
187447041
97111201
111132717
163767205
105781865
117913042
159631558
105821779
123813078
148999823
115460410
169944463
205905262
83596791
125456828
146524079
114583159
132225519
170888359
67084066
125459692
152429056
119020938
196270335
233049259
69671582
134295111
97460918
142160150
17282900
51215704
61177763
108

## Run a 'join' query to check table relationship set up correctly

In [21]:
# Create query to join Metadata and Data tables on ticker_id,
# and return data from both tables
open = select(Metadata.ticker_id, Metadata.symbol, Data.open).select_from(Metadata)\
        .join(Data, Metadata.ticker_id == Data.ticker_id)

In [22]:
# Execute query
results = session.execute(open)

In [23]:
# Print results
for row in results:
    print(row)

(0, 'AAPL', 109.11)
(0, 'AAPL', 117.64)
(0, 'AAPL', 121.01)
(0, 'AAPL', 123.66)
(0, 'AAPL', 123.75)
(0, 'AAPL', 125.08)
(0, 'AAPL', 130.28)
(0, 'AAPL', 132.04)
(0, 'AAPL', 132.76)
(0, 'AAPL', 133.52)
(0, 'AAPL', 133.75)
(0, 'AAPL', 136.04)
(0, 'AAPL', 136.6)
(0, 'AAPL', 138.21)
(0, 'AAPL', 141.9)
(0, 'AAPL', 143.97)
(0, 'AAPL', 146.36)
(0, 'AAPL', 146.83)
(0, 'AAPL', 148.21)
(0, 'AAPL', 148.985)
(0, 'AAPL', 149.9)
(0, 'AAPL', 152.83)
(0, 'AAPL', 154.89)
(0, 'AAPL', 155.08)
(0, 'AAPL', 156.64)
(0, 'AAPL', 156.71)
(0, 'AAPL', 161.01)
(0, 'AAPL', 164.27)
(0, 'AAPL', 164.695)
(0, 'AAPL', 166.96)
(0, 'AAPL', 167.48)
(0, 'AAPL', 169.28)
(0, 'AAPL', 174.01)
(0, 'AAPL', 174.03)
(0, 'AAPL', 174.28)
(0, 'AAPL', 175.6)
(0, 'AAPL', 177.7)
(0, 'AAPL', 177.83)
(0, 'AAPL', 184.46)
(0, 'AAPL', 189.485)
(0, 'AAPL', 191.64)
(0, 'AAPL', 193.78)
(0, 'AAPL', 196.235)
(0, 'AAPL', 203.17)
(0, 'AAPL', 206.43)
(0, 'AAPL', 209.88)
(0, 'AAPL', 213.9)
(0, 'AAPL', 219.05)
(0, 'AAPL', 225.07)
(0, 'AAPL', 227.95)
(0

In [24]:
# Close the session
session.close()